# Data Cleaning - Kickstarter Dataset

This notebook performs initial data cleaning on the Kickstarter dataset (df2).

**Steps:**
1. Load raw data
2. Drop unnecessary columns
3. Filter target states
4. Save cleaned datasets

**Outputs:**
- `kickstarter_cleaned.csv` - only "successful" and "failed"
- `kickstarter_cleaned_with_cancelled.csv` - includes "cancelled"

In [7]:
import pandas as pd
import numpy as np
from pathlib import Path

# Paths
BASE_DIR = Path.cwd().resolve().parents[1]
RAW_PATH = BASE_DIR / "data" / "raw"
CLEANED_PATH = BASE_DIR / "data" / "cleaned"

RAW_DATA_PATH = Path(RAW_PATH)
CLEANED_DATA_PATH = Path(CLEANED_PATH)

# Create output directory if not exists
CLEANED_DATA_PATH.mkdir(parents=True, exist_ok=True)

print("Setup complete")

Setup complete


We checked both datasets from 2016 and 2018. 
* There is considerable overlap but no discernable differences 
* 2018 contains more data

We decided to use only the 2018 dataset 

## Load Raw Data

In [8]:
# Load df2 (the larger/newer dataset)
# Adjust filename as needed
df = pd.read_csv(RAW_DATA_PATH / 'ks-projects-201801.csv')

print(f"Loaded: {df.shape[0]:,} rows, {df.shape[1]} columns")
print(f"\nColumns: {df.columns.tolist()}")

Loaded: 378,661 rows, 15 columns

Columns: ['ID', 'name', 'category', 'main_category', 'currency', 'deadline', 'goal', 'launched', 'pledged', 'state', 'backers', 'country', 'usd pledged', 'usd_pledged_real', 'usd_goal_real']


| Columns | Meaning | 
|:--------:|:--------:|
|  ID  |  Unique identifier   |  
|  name   |  Project title  | 
|  category |  Sub-category  |  
| main_category |  Main category |
|  currency |  Currency used for the project  | 
|  deadline |  Last day project backers can contribute | 
|  goal |  Target amount of money to be raised   | 
|  launched |  Project launch date  | 
|  pledged |  Amount of money people initially pledged  | 
|  state |  Project status (successful, canceled, failed, etc.)  | 
|  backers |  Number of people who supported the project  | 
|  country |  Country of the project creator   | 
|  usd pledged |  The 'pledged' column converted to USD (with missing data)  | 
|  usd_pledged_real |  Actual amount of money raised in USD  | 
|  usd_goal_real |  The 'goal' column converted to USD  | 

In [9]:
# Quick overview
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                378661 non-null  int64  
 1   name              378657 non-null  object 
 2   category          378661 non-null  object 
 3   main_category     378661 non-null  object 
 4   currency          378661 non-null  object 
 5   deadline          378661 non-null  object 
 6   goal              378661 non-null  float64
 7   launched          378661 non-null  object 
 8   pledged           378661 non-null  float64
 9   state             378661 non-null  object 
 10  backers           378661 non-null  int64  
 11  country           378661 non-null  object 
 12  usd pledged       374864 non-null  float64
 13  usd_pledged_real  378661 non-null  float64
 14  usd_goal_real     378661 non-null  float64
dtypes: float64(5), int64(2), object(8)
memory usage: 43.3+ MB


In [10]:
# Check state distribution before cleaning
print("State distribution (before):")
print(df['state'].value_counts())
print(f"\nTotal: {len(df):,}")

State distribution (before):
state
failed        197719
successful    133956
canceled       38779
undefined       3562
live            2799
suspended       1846
Name: count, dtype: int64

Total: 378,661


With the consolidated dataset, we do the initial cleaning of code. We drop the most obvious candidates which won't help the upcoming analysis.

The rest is refined iteratively later. 

## Drop Unnecessary Columns

**Dropping:**
- `name` - text, not needed for initial model
- `category` - too granular, keeping `main_category`
- `goal` - keeping `usd_goal_real` (normalized)
- `pledged` - keeping `usd_pledged_real` (normalized)
- `currency` - not needed since we use USD columns
- `usd pledged` - Kickstarter's conversion, less accurate than `usd_pledged_real`

Reasoning: 
* we won't be able to work with the names (yet)
* we will only work with the main_category as it's less granular and sufficient 
* Goal, pledged and currency go together, we work only with usd for reasons of comparison
* usd_pledged seems redundant so we leave it out 

In [11]:
# Columns to drop
COLS_TO_DROP = [
    'name',
    'category',
    'goal',
    'pledged',
    'currency',
    'usd pledged',
]

# Check which columns exist before dropping
existing_to_drop = [col for col in COLS_TO_DROP if col in df.columns]
missing = [col for col in COLS_TO_DROP if col not in df.columns]

print(f"Will drop: {existing_to_drop}")
if missing:
    print(f"Not found (skipping): {missing}")

Will drop: ['name', 'category', 'goal', 'pledged', 'currency', 'usd pledged']


In [12]:
# Drop columns
df_clean = df.drop(columns=existing_to_drop)

print(f"Columns before: {len(df.columns)}")
print(f"Columns after: {len(df_clean.columns)}")
print(f"\nRemaining columns: {df_clean.columns.tolist()}")

Columns before: 15
Columns after: 9

Remaining columns: ['ID', 'main_category', 'deadline', 'launched', 'state', 'backers', 'country', 'usd_pledged_real', 'usd_goal_real']


We want to be able to use durations and manipulate dates, so we transform the strings into regular datetime-columns.

In [13]:
# Transform dates into datetime
# deadline, launched - should be datetime types
df_clean["launched"] = pd.to_datetime(df_clean["launched"], errors="coerce")
df_clean["deadline"] = pd.to_datetime(df_clean["deadline"], errors="coerce")

# create new column - duration_days
df_clean["duration_days"] = (
    df_clean["deadline"] - df_clean["launched"]
).dt.days

For ease of use, we transform the category names to have homogenous formats.

In [14]:
# Columns clear naming
df_clean.columns = df_clean.columns.str.strip().str.replace(' ', '_').str.lower()
print(f"\nRenamed columns: {df_clean.columns.tolist()}")


Renamed columns: ['id', 'main_category', 'deadline', 'launched', 'state', 'backers', 'country', 'usd_pledged_real', 'usd_goal_real', 'duration_days']


## Filter Target States

**Keeping:**
- `successful` ✅
- `failed` ✅
- `canceled` (optional dataset)

**Dropping:**
- `live` - still running
- `suspended` - removed by Kickstarter
- `undefined` - unknown state

Reasoning: 
* There are only a few projects in "undefined" state. 
* For prediction reasons, those don't provide additional information as they are neither failed nor sucessful
* We are considering keeping "cancelled" as it's a type of failed 
* It would be fair to drop "cancelled" as it's self-initiated and constitutes a different set of probabilities than "failed for external reasons". 

For the second half, we initially kept two datasets with "cancelled" and without.

In [15]:
# Define states
MAIN_STATES = ['successful', 'failed']
OPTIONAL_STATE = 'canceled'

# Check current distribution
print("Current state distribution:")
print(df_clean['state'].value_counts())

Current state distribution:
state
failed        197719
successful    133956
canceled       38779
undefined       3562
live            2799
suspended       1846
Name: count, dtype: int64


In [16]:
# Dataset 1: Only successful & failed
df_main = df_clean[df_clean['state'].isin(MAIN_STATES)].copy()

print(f"Main dataset (successful + failed):")
print(f"  Rows: {len(df_main):,}")
print(f"  Distribution:")
print(df_main['state'].value_counts())

Main dataset (successful + failed):
  Rows: 331,675
  Distribution:
state
failed        197719
successful    133956
Name: count, dtype: int64


In [17]:
# Dataset 2: Including cancelled
states_with_cancelled = MAIN_STATES + [OPTIONAL_STATE]
df_with_cancelled = df_clean[df_clean['state'].isin(states_with_cancelled)].copy()

print(f"Dataset with cancelled:")
print(f"  Rows: {len(df_with_cancelled):,}")
print(f"  Distribution:")
print(df_with_cancelled['state'].value_counts())

Dataset with cancelled:
  Rows: 370,454
  Distribution:
state
failed        197719
successful    133956
canceled       38779
Name: count, dtype: int64


In [18]:
# For main dataset: successful=1, failed=0
df_main['target'] = (df_main['state'] == 'successful').astype(int)

# Drop state columns
df_main = df_main.drop(columns=['state'])

print("Main dataset target distribution:")
print(df_main['target'].value_counts())

Main dataset target distribution:
target
0    197719
1    133956
Name: count, dtype: int64


In [19]:
# For dataset with cancelled: successful=1, failed/canceled=0
df_with_cancelled['target'] = (df_with_cancelled['state'] == 'successful').astype(int)


# Drop state columns
df_with_cancelled = df_with_cancelled.drop(columns=['state'])

print("Dataset with cancelled - target distribution:")
print(df_with_cancelled['target'].value_counts())

Dataset with cancelled - target distribution:
target
0    236498
1    133956
Name: count, dtype: int64


## Save Cleaned Datasets

This data can be imported regularly into further notebooks:

In [20]:
# Save main dataset
main_path = CLEANED_DATA_PATH / 'kickstarter_cleaned.csv'
df_main.to_csv(main_path, index=False)
print(f" Saved: {main_path}")

# Save dataset with cancelled
cancelled_path = CLEANED_DATA_PATH / 'kickstarter_cleaned_with_cancelled.csv'
df_with_cancelled.to_csv(cancelled_path, index=False)
print(f"\n Saved: {cancelled_path}")

 Saved: D:\Programming\ai_ds_bootcamp\ds-ml-project_kickstarters\data\cleaned\kickstarter_cleaned.csv

 Saved: D:\Programming\ai_ds_bootcamp\ds-ml-project_kickstarters\data\cleaned\kickstarter_cleaned_with_cancelled.csv


## Summary

In [21]:
print("="*60)
print("DATA CLEANING COMPLETE")
print("="*60)
print(f"""
Original dataset:
  - Rows: {len(df):,}
  - Columns: {len(df.columns)}

Dropped columns:
  - {existing_to_drop}

Main dataset (kickstarter_cleaned.csv):
  - States: successful, failed
  - Rows: {len(df_main):,}

With cancelled (kickstarter_cleaned_with_cancelled.csv):
  - States: successful, failed, canceled
  - Rows: {len(df_with_cancelled):,}

Output location: {CLEANED_DATA_PATH.absolute()}
""")

DATA CLEANING COMPLETE

Original dataset:
  - Rows: 378,661
  - Columns: 15

Dropped columns:
  - ['name', 'category', 'goal', 'pledged', 'currency', 'usd pledged']

Main dataset (kickstarter_cleaned.csv):
  - States: successful, failed
  - Rows: 331,675

With cancelled (kickstarter_cleaned_with_cancelled.csv):
  - States: successful, failed, canceled
  - Rows: 370,454

Output location: D:\Programming\ai_ds_bootcamp\ds-ml-project_kickstarters\data\cleaned

